In [1]:
import numpy as np
import time
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, random_split
import torch.optim as optim
import matplotlib.pyplot as plt
import sys, os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(project_root)

from Resources.Model import Model_v7, Model_v8
from Resources.Game import *


In [2]:
'''
v7: 0 - 10610 

'''

'\nv7: 0 - 10610 \n\n'

In [22]:
# gamma = 0.98
# value_diff_scale = 50

# error_hist = []
# game_count = 0
# draw_count = 0

# criterion = nn.MSELoss()
# model = Model_v7() # Conv large
# model.train()
# # model.load_state_dict(torch.load('../Monte Carlo//Model Saves MC v4/model_36510_games'))
# white_wins = 0
# black_wins = 0
# draws = 0

# # for each game, save the average difference in value per move. Unpredictable random game should jump a lot.
# # Reasoable game should be stable or show slow conversions.
# avr_value_jump_hist = []

In [23]:
# Model Structure and number of trainable parameters

# print(sum(p.numel() for p in model.parameters() if p.requires_grad))
# for p in model.parameters():
#     if p.requires_grad:
#         print(p.shape, p.numel())

In [37]:

value_diff_scale = 250

test_count = 0

test_t0 = time.time()

# i_limit = 1000
while True:
    t0 = time.time()

    game = Game()
    next_root = None
    i = 0
    boards_white = [];  boards_black = []
    mat_white = [];     mat_black = []
    value_white = [];   value_black = []

    curr_value_jump_tot_white = 0
    curr_value_jump_tot_black = 0

    model.eval()

    while not game.is_over():
        i += 1
        if i > i_limit:
            break

        current_boards = []; current_labels = []
    
        moves = game.PossibleMoves()

        game_ini = game.copy()
        board_batch = [board_to_tensor(game.pieces)]

        mate = False

        for move in moves:
            game.PlayMove(move)
            board_batch.append(board_to_tensor(game.pieces))
            game.FlipBoard()
            if game.is_over():
                mate = True
                chosen_move = move
                game = game_ini.copy()
                break
            game = game_ini.copy()

        if not mate:
            board_tensor = torch.stack(board_batch)
            values = model(board_tensor)
            # if i < 11:
            #     scale = value_diff_scale/15
            # elif i < 21:
            #     scale = value_diff_scale/3
            # else:
            #     scale = value_diff_scale*3
            scale = value_diff_scale
            values_diff = [scale*(values[i] - values[0]) for i in range(1, len(values))]
            move_prob = torch.softmax(torch.Tensor(values_diff), dim=0).numpy()
            # print(move_prob)
            chosen_i = np.random.choice(range(len(moves)), p=move_prob)
            chosen_move = moves[chosen_i]
            
        game.PlayMove(chosen_move)

        mat_diff = game.MaterialDiff()
        value = model(torch.stack([board_to_tensor(game.pieces)])).detach().item()

        if i % 2 == 1:
            boards_white.append(board_to_tensor(game.pieces))
            mat_white.append(mat_diff)
            value_white.append(value)
            
            if i > 1:
                curr_value_jump_tot_white += abs(value_white[-1] - value_white[-2])

        if i % 2 == 0:
            boards_black.append(board_to_tensor(game.pieces))
            mat_black.append(mat_diff)
            value_black.append(value)

            if i > 2:
                curr_value_jump_tot_black += abs(value_black[-1] - value_black[-2])

        game.FlipBoard()

    test_count += 1

    if test_count == 100:
        print('time for 100 game = ', time.time() - test_t0)
        break
    else:
        continue

    avr_value_jump_hist.append( (curr_value_jump_tot_black + curr_value_jump_tot_white) / (i-2) )

    if True: #i <= i_limit: # game actually ended
        winner = game.get_winner()
        if winner == 'draw':
            draws += 1;         reward_white = 0;   reward_black = 0
        if winner == 'white':
            white_wins += 1;    reward_white = 1;   reward_black = -1
        if winner == 'black':
            black_wins += 1;    reward_white = -1;  reward_black = 1

    # else: # game terminated because too long
    #     matdiff = game.MaterialDiff()
    #     if matdiff > 4:
    #         if game.turn == 'white':
    #             winner = 'white'; reward_white = 0.5;   reward_black = -0.5;    white_wins += 0.5
    #         if game.turn == 'black':
    #             winner = 'black'; reward_white = -0.5;  reward_black = 0.5;     black_wins = 0.5
    #     elif matdiff < -4:
    #         if game.turn == 'white':
    #             winner = 'black'; reward_white = -0.5;  reward_black = 0.5;     black_wins += 0.5
    #         if game.turn == 'black':
    #             winner = 'white'; reward_white = 0.5;   reward_black = -0.5;    white_wins = 0.5
    #     else:
    #         winner = 'draw'; draws += 1; reward_white = 0; reward_black = 0

    # if i > i_limit:
    #     print('game termianted, mat diff = ', matdiff)

    game_time = time.time() - t0
    # print('game_time: ', game_time, 'i max: ', i)

    labels_white = [reward_white * gamma**(len(boards_white) - 1 - i) for i in range(len(boards_white))]
    labels_black = [reward_black * gamma**(len(boards_black) - 1 - i) for i in range(len(boards_black))]

    inputs = boards_white + boards_black
    labels = labels_white + labels_black

    inputs_tens = torch.stack(inputs)
    labels_tens = torch.Tensor(labels)   

    if winner != 'draw':
        # newest training error
        out = model(inputs_tens)
        out = out.view(out.shape[0])
        loss = criterion(out, labels_tens)
        error_hist.append(loss.item())
    else:
        draw_count += 1
        continue

    if game_count % 10 == 0:
        print(' -- {} -- winner: {}, i: {}   '.format(game_count, winner, i), 'wins: w = {}, b = {}, d = {}'.format(white_wins, black_wins, draws))

    if game_count % 50 == 1:# or True:
        plt.figure(figsize=(6, 3))
        plt.plot(mat_white, label='mat white')
        plt.legend()
        plt.show()
        plt.figure(figsize=(6, 3))
        plt.plot(value_white, label='value white')
        plt.plot(value_black, label='value black')
        plt.legend()
        plt.show()
        plt.figure(figsize=(6, 3))
        plt.plot([np.mean(avr_value_jump_hist[i:i+400]) for i in range(len(avr_value_jump_hist) - 400)], label='average value jumps, rolling mean(400)')
        plt.legend()
        plt.show()

    torch.save(inputs_tens, '/Users/Philip/Desktop/Projects/RL Chess/MCTS/Game Saves v7/inputs_{}'.format(game_count))
    torch.save(labels_tens, '/Users/Philip/Desktop/Projects/RL Chess/MCTS/Game Saves v7/labels_{}'.format(game_count))

    t1 = time.time()

    if game_count > 5:
        # sample some of previous game for replay. More recent -> more likely
        decay_rate = 0.003
        prob =  np.exp(-decay_rate * np.arange(game_count)) 
        prob = prob /sum(prob)
        num_samples = 3
        samples = np.random.choice(np.arange(game_count, 0, -1), size=num_samples, p=prob)

        for indices in samples:
            inputs_load = torch.load('/Users/Philip/Desktop/Projects/RL Chess/MCTS/Game Saves v7/inputs_{}'.format(indices))
            inputs_tens = torch.cat((inputs_load, inputs_tens))
            labels_tens = torch.cat((torch.load('/Users/Philip/Desktop/Projects/RL Chess/MCTS/Game Saves v7/labels_{}'.format(indices)), labels_tens))

    len_boards = len(inputs_tens)
    for c in range(len_boards):
        tens = inputs_tens[c]
        # if no pawns are on the board: add all other 3 rotated versions of the board to the current data set
        if torch.sum(inputs_tens[c][0]) + torch.sum(inputs_tens[c][6]) == 0:
            board_new = tensor_to_board(tens)
            for _ in range(3):
                board_new = rotate_board(board_new, 1)
                tens_new = torch.stack([board_to_tensor(board_new)])
                inputs_tens = torch.cat((inputs_tens, tens_new), 0)
                labels_tens = torch.cat((labels_tens, torch.stack([labels_tens[c]])), 0)
    
    model.train()
    # learning_rate = 5e-4 * 500 / (500 + game_count) # reducing learning rate, 1/n one possible options
    learning_rate = 5e-5
    # weight_decay = 1e-2 # regularization to avoid overfitting
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    random_order = [i for i in range(len(inputs_tens))]
    np.random.shuffle(random_order)
    for c in random_order:

        inp = torch.stack([inputs_tens[c], mirror_board_tensor(inputs_tens[c])])
        label = torch.stack([labels_tens[c], labels_tens[c]])

        out = model(inp)
        out = out.view(out.shape[0])

        loss = criterion(out, label)
        optimizer.zero_grad()
        loss.backward()

        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_gradient_norm)
        optimizer.step()

    # print('game time: {} -- training time: {},  '.format(game_time, time.time() - t1))
    # print('sum of losses: ', sum(err_curr))

    # error_hist.append(sum(err_curr))
    
    torch.save(error_hist, './error_hist MC v7')

    if game_count % 100  == 1:
        # plt.plot(error_hist)
        # plt.title('training errors')
        # plt.show()
        interval = 400
        plt.figure(figsize=(6, 3))
        plt.plot([np.mean(error_hist[i:i+interval]) for i in range(len(error_hist) - interval)], label='error hist, rolling mean '+str(interval))
        plt.show()

    if game_count % 100 == 9:    
        torch.save(model.state_dict(), './Model Saves MC v7/model_{}_games'.format(game_count+1))

    game_count += 1


time for 100 game =  61.40547800064087


In [4]:
model = Model_v7()
print(sum(i.numel() for i in model.parameters()))

model = Model_v8()
print(sum(i.numel() for i in model.parameters()))




95813
2188757
